In [49]:
import sys
import math,random
import time
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import (QCoreApplication, QPropertyAnimation, QDate, QDateTime, QMetaObject, QObject, QPoint,
    QRect, QSize, QTime, QUrl, Qt, QEvent)


from PyQt5.QtGui import (QBrush, QColor, QConicalGradient, QCursor, QFont,
    QFontDatabase, QIcon, QKeySequence, QLinearGradient, QPalette, QPainter, QPixmap,
    QRadialGradient)
from PyQt5.QtWidgets import QMainWindow
from PyQt5.QtWidgets import *
import pymysql as p
import re

In [91]:
#Splash Screen
from ui_splash_screen import Ui_MainWindow
# login
import ui_login
#sign up
from ui_Register import Ui_SignUpWindow
#Dashboard
import ui_DashboardNew
#video upload
from ui_VideoUploadNew import Ui_Welcome
#Video Analysis
from VideoFileAnalysis import VideoAnalyzer
#Live Feed
import ui_LiveFeedNew
#Send Email
from SendEmail import smtp_gmail
#forgot pwd dailog box
import ui_ForgotPassword 
#show history
import ui_History
#choose history type
import ui_ActivityHistory

#global var
mysql_password = 'prakhars'
counter = 0
userID=''
history = {}
file_type=1

In [92]:
# Generate OTP
def generateOTP() :
    string = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    OTP = ""
    length = len(string)
    for i in range(6) :
        OTP += string[math.floor(random.random() * length)]
  
    return OTP

def NotValid(email,pwd):
    regex = '^(\w|\.|\-|\_)+[@](\w|\_|\-|\.)+[.]\w{2,3}$' #/^[a-zA-Z._]{3,}@[a-zA-Z]{3,}[.]{1}[a-zA-Z]{2,}$/
    regex_pwd = '^(\w|\_|\@){8}'
    if(re.search(regex, email) and re.search(regex_pwd, pwd)):
        return False
    return True

def SendMail(result):
    
    checkString='No Suspicious Activity found in Video'
        
    if checkString not in result:
        global userID
        subject="Alert!! Suspicious Activity Detected"
        con=p.connect(host='localhost',user='root',password=mysql_password,db='analysis',charset='utf8mb4',cursorclass=p.cursors.DictCursor)
        cur=con.cursor()
        q="select name from sign_up where emailid='%s'"
        cur.execute(q %(userID))
        data=cur.fetchone()
        user_name=data['name']
        con.commit()
        con.close()
        mail_body="Dear " + str(user_name) + "\n\nA risk was generated by Suspicious Activity Detector that requires your attention.\n\n" + result + "\n\n\nRegards\nThe Suspicion Detection Team "
        
        smtp_gmail(mail_body,userID,subject)
        
def get_record(Type):
    
    con=p.connect(host='localhost',user='root',password=mysql_password,db='analysis',charset='utf8mb4',cursorclass=p.cursors.DictCursor)
    cur=con.cursor()
    
    if Type == 1:
        q="select date,filename,timestamps from videofile_history where emailid='%s'";
    else:
        q="select date,timestamps from videofeed_history where emailid='%s'";
        
    cur.execute(q %(userID))
    data=cur.fetchall()
        
    
    con.commit()
    con.close()
    return data
    
def save_record(res,filename,Type):
    
    con=p.connect(host='localhost',user='root',password=mysql_password,db='analysis',charset='utf8mb4',cursorclass=p.cursors.DictCursor)
    cur=con.cursor()
    output = eval(res)
    output=' '.join(output)
    if Type == 1:
        q="insert into videofile_history (emailid,date,filename,timestamps) values('%s',curdate(),'%s','%s')";
        cur.execute(q %(userID,filename,output))
        
    else:
        q="insert into videofeed_history (emailid,date,timestamps) values('%s',curdate(),'%s')";
        cur.execute(q %(userID,output))
    
    con.commit()
    con.close()
        

In [93]:
class LiveFeedWindow(QMainWindow, ui_LiveFeedNew.Ui_Welcome):
    def __init__(self,parent=None):
        QMainWindow.__init__(self)
        super(LiveFeedWindow, self).__init__(parent)
        self.setupUi(self)
        self.setWindowIcon(QtGui.QIcon("robo.gif"))
        self.setFixedSize(810, 598)
        
        global userID 
        l=userID.split('@')
        self.username.setText("<strong>Welcome </strong>"+str(l[0]))
        
        self.StopAlarm.hide()
        self.Start.clicked.connect(self.StartAnalysis)
        self.back.clicked.connect(self.Back)
        
       
        
    def StartAnalysis(self):
        
        self.update.setText("Analyzing........Please wait !!")
        self.output.setText("Press 'q' to Stop Analyzing...")
        
        result = self.VideoFeedAnalysis()
        
        self.update.setText("")
        self.output.setText(result)
        
        index=result.find('[')
        res=result[index:]
        save_record(res,'',0)
        
        if ui_DashboardNew.EMail==1: 
            self.update.setText("") #initial text
            
            #change text
            QtCore.QTimer.singleShot(100, lambda: self.update.setText("<strong>Sending Mail</strong>"))
            QtCore.QTimer.singleShot(6000, lambda: self.update.setText("Press Start to start again.."))
            
            try:
                SendMail(result)
                
            except:
                self.update.setText("") #initial text
            
                #change text
                QtCore.QTimer.singleShot(100, lambda: self.update.setText("<strong>No Internet Connection Found! Mail was not sent!</strong>"))
                QtCore.QTimer.singleShot(8000, lambda: self.update.setText("Press Start to start again.."))
            
        else:
            self.update.setText("Press Start to start again..")
            self.output.setText(result)
        
        QtCore.QTimer.singleShot(10000, lambda: self.update.setText("Press Start to start again.."))
        
            
        
    def Back(self):
        
        #show dashboard window
        self.main=UserDashboard()
        self.main.show()
            
        #close live feed window
        self.close()

In [94]:
class VideoFileWindow(QMainWindow, Ui_Welcome):
    def __init__(self,parent=None):
        QMainWindow.__init__(self)
        super(VideoFileWindow, self).__init__(parent)
        self.setupUi(self)
        self.setWindowIcon(QtGui.QIcon("robo.gif"))
        self.setFixedSize(900, 617)
        
        #self.video_path=None
        self.label_drag.setText("<strong> DRAG FILES HERE </strong>") #initial text
            
        #change text
        #QtCore.QTimer.singleShot(100, lambda: self.label_drag.setText("<strong> DRAG FILES HERE </strong>"))
        QtCore.QTimer.singleShot(3000, lambda: self.label_drag.setText(""))

        global userID 
        l=userID.split('@')
        self.username.setText("<strong>Welcome </strong>"+str(l[0]))
        
        self.copy_path.clicked.connect(self.copyPathFunction)
        
        self.Start.clicked.connect(self.Analysis)  #Video Analysis Start
        
        self.back.clicked.connect(self.Back)
      
        
    def copyPathFunction(self):
        
        self.video_path=self.listWidget.currentItem()
                
        if self.video_path==None:
            
            self.PathCopiedMsg.setText("") #initial text
            
            #change text
            QtCore.QTimer.singleShot(100, lambda: self.PathCopiedMsg.setText("<strong>No File Selected ! </strong>"))
            QtCore.QTimer.singleShot(4000, lambda: self.PathCopiedMsg.setText(""))
        
        else:
            
            self.PathCopiedMsg.setText("") #initial text
            
            #change text
            QtCore.QTimer.singleShot(100, lambda: self.PathCopiedMsg.setText("<strong>Path Copied ! </strong>"))
            QtCore.QTimer.singleShot(3000, lambda: self.PathCopiedMsg.setText(""))
            
            self.video_path=self.video_path.text()
        
    #analysis part
    def Analysis(self):
        
        List=(self.video_path.split('.'))
        
        if List[-1] in ['mp4','mov','avi'] :
            
            self.output.setText("Press 'q' to stop Analyzing...")
            result=VideoAnalyzer(self.video_path)

            #display timestamp and output

            self.output.setText(result) 
            
            index=result.find('[')
            res=result[index:]
            save_record(res,self.video_path,1)

            if ui_DashboardNew.EMail==1: 
                
                try:
                    SendMail(result)
                except:
                    self.PathCopiedMsg.setText("") #initial text
            
                    #change text
                    QtCore.QTimer.singleShot(100, lambda: self.PathCopiedMsg.setText("<strong>No Internet Connection Found! Mail was not sent!</strong>"))
                    QtCore.QTimer.singleShot(8000, lambda: self.PathCopiedMsg.setText(""))
                    
        
        else :    
            self.PathCopiedMsg.setText("") #initial text
            
            #change text
            QtCore.QTimer.singleShot(100, lambda: self.PathCopiedMsg.setText("<strong>Invalid Video File Format !</strong>"))
            QtCore.QTimer.singleShot(5000, lambda: self.PathCopiedMsg.setText(""))
            
    def Back(self):
        
        #show dashboard window
        self.main=UserDashboard()
        self.main.show()
            
        #close video analysis window
        self.close()
            
        
        

In [95]:
##Dashboard...

class UserDashboard(QMainWindow, ui_DashboardNew.Ui_Welcome):
    def __init__(self,parent=None):
        QMainWindow.__init__(self)
        super(UserDashboard, self).__init__(parent)
        self.setupUi(self)
        self.setWindowIcon(QtGui.QIcon("robo.gif"))
        self.setFixedSize(882, 672)
        
        global userID 
        l=userID.split('@')
        self.username.setText("<strong>Welcome </strong>"+str(l[0]))
      
        self.liveFeed.clicked.connect(self.OnClickLiveFeed)
        
        self.videoAnalysis.clicked.connect(self.OnClickVideoFile)
        
        self.logout.clicked.connect(self.Logout)
        
        self.history.clicked.connect(self.ShowHistory)
            
        
    def OnClickVideoFile(self):
        
        #show upload video window
        self.main=VideoFileWindow()
        self.main.show()
            
        #close dashboard window
        self.close()
        
    def OnClickLiveFeed(self):
        
        #show live feed window
        self.main=LiveFeedWindow()
        self.main.show()
            
        #close dashboard window
        self.close()
        
    def ShowHistory(self):
        
        #show ActivityHistory 
        self.main=ChooseActivityHistory()
        self.main.show()
            
        #close dashboard window
        self.close()
        
        
    def Logout(self):
        
        userID=''
        #show login window
        self.main=LoginWindow()
        self.main.show()

        #close dashboard window
        self.close()
        
        

In [96]:
#choose type- history
class ChooseActivityHistory(QMainWindow, ui_ActivityHistory.Ui_Welcome):
    
    def __init__(self,parent=None):
        QMainWindow.__init__(self)
        super(ChooseActivityHistory, self).__init__(parent)
        self.setupUi(self)
        self.setWindowIcon(QtGui.QIcon("robo.gif"))
        self.setFixedSize(882, 672)
        
        global userID 
        l=userID.split('@')
        self.username.setText("<strong>Welcome </strong>"+str(l[0]))
        
        self.back.clicked.connect(self.Back)
        
        self.videofeed.clicked.connect(self.OnClickLiveFeed)
        
        self.videofile.clicked.connect(self.OnClickVideoFile)
        
    def OnClickVideoFile(self):
        
        global file_type
        file_type=1
        #show history
        self.main=ShowHistory()
        self.main.show()
            
        #close dashboard window
        self.close()
        
    def OnClickLiveFeed(self):
        
        global file_type
        file_type=0
        #show history
        self.main=ShowHistory()
        self.main.show()
            
        #close history type
        self.close()
        
    def Back(self):
        
        #show dashboard window
        self.main=UserDashboard()
        self.main.show()
            
        #close video analysis window
        self.close()

In [97]:
#show history
class ShowHistory(QMainWindow, ui_History.Ui_MainWindow):
    
    def __init__(self,parent=None):
        QMainWindow.__init__(self)
        super(ShowHistory, self).__init__(parent)
        self.setupUi(self)
        self.setWindowIcon(QtGui.QIcon("robo.gif"))
        self.setFixedSize(874, 575)
        
        global userID 
        l=userID.split('@')
        self.username.setText("<strong>Welcome </strong>"+str(l[0]))
        
        self.CreateTable()
        
        self.table.setColumnWidth(0,50)
        self.table.setColumnWidth(1,100)
        self.table.setColumnWidth(2,150)
        
        self.back.clicked.connect(self.Back)
        
    def CreateTable(self):

        global file_type
        #Column count
        if file_type == 1: 
            record = get_record(1)
            self.table.setRowCount(len(record))
            self.table.setColumnCount(3)
            self.table.setHorizontalHeaderLabels(("Date","File Name","Timestamps"))
            i=0
            while(i<len(record)):
                self.table.setItem(i,0, QTableWidgetItem(str(record[i]['date'])))
                self.table.setItem(i,1, QTableWidgetItem(str(record[i]['filename'])))
                self.table.setItem(i,2, QTableWidgetItem(str(record[i]['timestamps'])))
                i+=1
                
            
        else: 
            record = get_record(0)
            self.table.setRowCount(len(record))
            self.table.setColumnCount(2)
            self.table.setHorizontalHeaderLabels(("Date","Timestamps"))
           
            i=0
            while(i<len(record)):
                self.table.setItem(i,0, QTableWidgetItem(str(record[i]['date'])))
                self.table.setItem(i,1, QTableWidgetItem(str(record[i]['timestamps'])))
                i+=1



        #Table will fit the screen horizontally
        self.table.horizontalHeader().setStretchLastSection(True)
        self.table.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)

        
    def Back(self):
        
        #show ChooseActivityHistory
        self.main=ChooseActivityHistory()
        self.main.show()
            
        #close show history
        self.close()
    


In [98]:
##Forgot Password
class ForgotPassword(QDialog):
    def __init__(self,parent=None):
        super().__init__()
        self.setupUi(self)
        self.setWindowTitle("Reset Password")
        self.setWindowIcon(QtGui.QIcon("robo.gif"))
        self.setFixedSize(503, 394)
        
        self.otp=""
        self.send.clicked.connect(self.sendOTPmail)
        self.Ok.clicked.connect(self.okClicked)
        self.Cancel.clicked.connect(self.cancel)
        
        self.VerifyCode.setMaxLength(6)
        

    def setupUi(self, Dialog):
        if Dialog.objectName():
            Dialog.setObjectName(u"Dialog")
        Dialog.resize(503, 394)
        Dialog.setFocusPolicy(Qt.StrongFocus)
        Dialog.setStyleSheet(u"	background-color: rgb(56, 58, 89);\n"
"	color: rgb(59, 61, 157);\n"
"   	border-radius:10px;\n"
"")
        self.frame = QFrame(Dialog)
        self.frame.setObjectName(u"frame")
        self.frame.setGeometry(QRect(20, 19, 461, 301))
        self.frame.setStyleSheet(u"QFrame{\n"
"	background-color: rgb(166, 144, 255);\n"
"	color: rgb(96, 98, 189);\n"
"	border-radius:10px;\n"
"}\n"
"")
        self.frame.setFrameShape(QFrame.StyledPanel)
        self.frame.setFrameShadow(QFrame.Raised)
        self.LabelNewPwd = QLabel(self.frame)
        self.LabelNewPwd.setObjectName(u"LabelNewPwd")
        self.LabelNewPwd.setGeometry(QRect(38, 195, 111, 41))
        font = QFont()
        font.setFamily(u"Segoe UI")
        font.setPointSize(11)
        self.LabelNewPwd.setFont(font)
        self.LabelNewPwd.setStyleSheet(u"color: rgb(56, 58, 89);\n"
"background:transparent;")
        self.LabelNewPwd.setAlignment(Qt.AlignCenter)
        self.LabelConfPwd = QLabel(self.frame)
        self.LabelConfPwd.setObjectName(u"LabelConfPwd")
        self.LabelConfPwd.setGeometry(QRect(33, 235, 131, 41))
        self.LabelConfPwd.setFont(font)
        self.LabelConfPwd.setStyleSheet(u"color: rgb(56, 58, 89);\n"
"background:transparent;")
        self.LabelConfPwd.setAlignment(Qt.AlignCenter)
        self.ConfirmPwd = QLineEdit(self.frame)
        self.ConfirmPwd.setObjectName(u"ConfirmPwd")
        self.ConfirmPwd.setGeometry(QRect(173, 245, 244, 23))
        font1 = QFont()
        font1.setFamily(u"Segoe UI")
        font1.setPointSize(12)
        self.ConfirmPwd.setFont(font1)
        self.ConfirmPwd.setFocusPolicy(Qt.StrongFocus)
        self.ConfirmPwd.setStyleSheet(u"border-radius:10px;\n"
"background-color: rgb(255, 255, 255);")
        self.ConfirmPwd.setMaxLength(8)
        self.ConfirmPwd.setEchoMode(QLineEdit.Password)
        self.ConfirmPwd.setAlignment(Qt.AlignCenter)
        self.ConfirmPwd.setClearButtonEnabled(True)
        self.NewPassword = QLineEdit(self.frame)
        self.NewPassword.setObjectName(u"NewPassword")
        self.NewPassword.setGeometry(QRect(173, 205, 244, 23))
        self.NewPassword.setFont(font1)
        self.NewPassword.setFocusPolicy(Qt.StrongFocus)
        self.NewPassword.setStyleSheet(u"border-radius:10px;\n"
"background-color: rgb(255, 255, 255);")
        self.NewPassword.setMaxLength(8)
        self.NewPassword.setEchoMode(QLineEdit.Password)
        self.NewPassword.setAlignment(Qt.AlignCenter)
        self.NewPassword.setClearButtonEnabled(True)
        self.VerifyCode = QLineEdit(self.frame)
        self.VerifyCode.setObjectName(u"VerifyCode")
        self.VerifyCode.setGeometry(QRect(178, 165, 244, 23))
        self.VerifyCode.setFont(font1)
        self.VerifyCode.setFocusPolicy(Qt.StrongFocus)
        self.VerifyCode.setAutoFillBackground(False)
        self.VerifyCode.setStyleSheet(u"border-radius:10px;\n"
"background-color: rgb(255, 255, 255);")
        self.VerifyCode.setAlignment(Qt.AlignCenter)
        self.VerifyCode.setClearButtonEnabled(True)
        self.EnterCode = QLabel(self.frame)
        self.EnterCode.setObjectName(u"EnterCode")
        self.EnterCode.setGeometry(QRect(38, 155, 121, 31))
        self.EnterCode.setFont(font)
        self.EnterCode.setStyleSheet(u"color: rgb(56, 58, 89);\n"
"background:transparent;")
        self.EnterCode.setAlignment(Qt.AlignCenter)
        self.Status = QLabel(self.frame)
        self.Status.setObjectName(u"Status")
        self.Status.setGeometry(QRect(60, 90, 341, 71))
        font2 = QFont()
        font2.setFamily(u"Segoe UI")
        font2.setPointSize(11)
        font2.setBold(True)
        font2.setWeight(75)
        font2.setKerning(True)
        self.Status.setFont(font2)
        self.Status.setStyleSheet(u"color: rgb(202, 5, 31);\n"
"background:transparent;")
        self.Status.setAlignment(Qt.AlignCenter)
        self.Status.setWordWrap(True)
        self.EmailId = QLineEdit(self.frame)
        self.EmailId.setObjectName(u"EmailId")
        self.EmailId.setGeometry(QRect(170, 20, 244, 23))
        self.EmailId.setFont(font1)
        self.EmailId.setFocusPolicy(Qt.StrongFocus)
        self.EmailId.setStyleSheet(u"border-radius:10px;\n"
"background-color: rgb(255, 255, 255);")
        self.EmailId.setEchoMode(QLineEdit.Normal)
        self.EmailId.setAlignment(Qt.AlignCenter)
        self.EmailId.setClearButtonEnabled(True)
        self.LabelEmail = QLabel(self.frame)
        self.LabelEmail.setObjectName(u"LabelEmail")
        self.LabelEmail.setGeometry(QRect(30, 13, 111, 41))
        self.LabelEmail.setFont(font)
        self.LabelEmail.setStyleSheet(u"color: rgb(56, 58, 89);\n"
"background:transparent;")
        self.LabelEmail.setAlignment(Qt.AlignCenter)
        self.send = QPushButton(self.frame)
        self.send.setObjectName(u"send")
        self.send.setGeometry(QRect(200, 60, 71, 31))
        font3 = QFont()
        font3.setFamily(u"Segoe UI")
        font3.setPointSize(12)
        font3.setBold(False)
        font3.setItalic(False)
        font3.setWeight(9)
        self.send.setFont(font3)
        self.send.setCursor(QCursor(Qt.PointingHandCursor))
        self.send.setMouseTracking(True)
        self.send.setStyleSheet(u"QPushButton{\n"
"border-radius:10px;\n"
"	background-color: rgb(56,58,89);\n"
"	color: rgb(255, 255, 255);\n"
"	font: 75 12pt \"Segoe UI\";\n"
"}\n"
"QPushButton:hover{\n"
"	\n"
"	color:  rgb(13,13,13);\n"
"	background-color: rgb(255, 171, 215);\n"
"	font: 90 12pt \"Segoe UI\";\n"
"}")
        self.Ok = QPushButton(Dialog)
        self.Ok.setObjectName(u"Ok")
        self.Ok.setGeometry(QRect(170, 350, 71, 31))
        self.Ok.setFont(font3)
        self.Ok.setCursor(QCursor(Qt.PointingHandCursor))
        self.Ok.setMouseTracking(True)
        self.Ok.enabled = False;
        self.Ok.setStyleSheet(u"QPushButton{\n"
"border-radius:10px;\n"
"	background-color: rgb(255, 171, 215);\n"
"	color: rgb(13,13,13);\n"
"	font: 75 12pt \"Segoe UI\";\n"
"}\n"
"QPushButton:hover{\n"
"	\n"
"	color:  rgb(255, 255, 255);\n"
"	background-color: rgb(166, 144, 255);\n"
"	font: 90 12pt \"Segoe UI\";\n"
"}")
        self.Cancel = QPushButton(Dialog)
        self.Cancel.setObjectName(u"Cancel")
        self.Cancel.setGeometry(QRect(260, 350, 71, 31))
        self.Cancel.setFont(font3)
        self.Cancel.setCursor(QCursor(Qt.PointingHandCursor))
        self.Cancel.setMouseTracking(True)
        self.Cancel.setStyleSheet(u"QPushButton{\n"
"border-radius:10px;\n"
"	background-color: rgb(255, 171, 215);\n"
"	color: rgb(13,13,13);\n"
"	font: 75 12pt \"Segoe UI\";\n"
"}\n"
"QPushButton:hover{\n"
"	\n"
"	color:  rgb(255, 255, 255);\n"
"	background-color: rgb(166, 144, 255);\n"
"	font: 90 12pt \"Segoe UI\";\n"
"}")

        self.retranslateUi(Dialog)

        QMetaObject.connectSlotsByName(Dialog)
    # setupUi

    def retranslateUi(self, Dialog):
        Dialog.setWindowTitle(QCoreApplication.translate("Dialog", u"Dialog", None))
        self.LabelNewPwd.setText(QCoreApplication.translate("Dialog", u"New Password", None))
        self.LabelConfPwd.setText(QCoreApplication.translate("Dialog", u"Confirm Password", None))
        self.ConfirmPwd.setText("")
        self.ConfirmPwd.setPlaceholderText(QCoreApplication.translate("Dialog", u"Confirm Password", None))
        self.NewPassword.setText("")
        self.NewPassword.setPlaceholderText(QCoreApplication.translate("Dialog", u"Password", None))
        self.VerifyCode.setText("")
        self.VerifyCode.setPlaceholderText(QCoreApplication.translate("Dialog", u"Enter Code", None))
        self.EnterCode.setText(QCoreApplication.translate("Dialog", u"Verification Code", None))
        self.Status.setText(QCoreApplication.translate("Dialog", u"Verification Code will be sent to your registered Email address", None))
        self.EmailId.setText("")
        self.EmailId.setPlaceholderText(QCoreApplication.translate("Dialog", u"Email-id", None))
        self.LabelEmail.setText(QCoreApplication.translate("Dialog", u"Email ID", None))
        self.send.setText(QCoreApplication.translate("Dialog", u"Send", None))
        self.Ok.setText(QCoreApplication.translate("Dialog", u"Submit", None))
        self.Cancel.setText(QCoreApplication.translate("Dialog", u"Cancel", None))
    # retranslateUi

    
    def sendOTPmail(self):
        
        self.email=self.EmailId.text()
        self.otp = generateOTP()
        
        regex = '^(\w|\.|\-|\_)+[@](\w|\_|\-|\.)+[.]\w{2,3}$';
        
        if self.email=="":
            
            self.Status.setText("") #initial text

            #change text
            QtCore.QTimer.singleShot(100, lambda: self.Status.setText("<strong>* Please Enter your Email-id !! *</strong>"))
            QtCore.QTimer.singleShot(3000, lambda: self.Status.setText(""))
            
        elif not (re.search(regex, self.email)):
              
            self.Status.setText("") #initial text

            #change text
            QtCore.QTimer.singleShot(100, lambda: self.Status.setText("<strong>* Please Enter Valid Email-id !! *</strong>"))
            QtCore.QTimer.singleShot(3000, lambda: self.Status.setText(""))
        
        else:
            
            subject="Password Reset Request";

            con=p.connect(host='localhost',user='root',password=mysql_password,db='analysis',charset='utf8mb4',cursorclass=p.cursors.DictCursor)
            cur=con.cursor()

            #check if email exist
            q1="select emailid from sign_up where exists (select name from sign_up where emailid='%s');"
            cur.execute(q1 %(self.email))
            data=cur.fetchone()

            if data==None:

                self.Status.setText("") #initial text

                #change text
                QtCore.QTimer.singleShot(100, lambda: self.Status.setText("<strong> Email-ID not registered !! </strong>"))
                QtCore.QTimer.singleShot(4000, lambda: self.Status.setText(""))

            else:

                q="select name from sign_up where emailid='%s'"
                cur.execute(q %(self.email))
                data=cur.fetchone()
                user_name = data['name']

                con.commit()
                con.close()

                self.Status.setText("Verification Code Sent!! Please check your mail")
                self.send.setText("Resend")

                mail_body="Hi " + str(user_name)+"," + "\n\nWe recieved a request to reset your password.\n\nUse the following verification code to reset your password.\n\n      *VERIFICATION CODE* : " + str(self.otp) + "\n\n\nRegards\n\nThe Suspicion Detection Team "
                
                try:
                    smtp_gmail(mail_body,self.email,subject)  
                
                except:
                    
                    self.Status.setText("") #initial text

                    #change text
                    QtCore.QTimer.singleShot(100, lambda: self.Status.setText("<strong> No Internet Connection Found !! </strong>"))
                    QtCore.QTimer.singleShot(4000, lambda: self.Status.setText(""))
                    
                
                
            
    def okClicked(self):
        
        Code = self.VerifyCode.text()
        
        pwd = self.NewPassword.text()
        confPwd = self.ConfirmPwd.text()
              
        regex_pwd = '^(\w|\_|\@){8}';
            
        if self.otp=="": #Send button was not clicked
            
            self.Status.setText("") #initial text

            #change text
            QtCore.QTimer.singleShot(100, lambda: self.Status.setText("<strong>* Please Enter your Email-id and Press the button !! *</strong>"))
            QtCore.QTimer.singleShot(3000, lambda: self.Status.setText(""))
            
        elif Code=="": #Verification code was not entered
            
            self.Status.setText("") #initial text

            #change text
            QtCore.QTimer.singleShot(100, lambda: self.Status.setText("<strong>* Please enter the Verification Code !! *</strong>"))
            QtCore.QTimer.singleShot(3000, lambda: self.Status.setText(""))
            
        elif (str(Code) != str(self.otp)):

            self.Status.setText("") #initial text

            #change text
            QtCore.QTimer.singleShot(100, lambda: self.Status.setText("<strong>* Incorrect Verification Code!! *</strong>"))
            QtCore.QTimer.singleShot(3000, lambda: self.Status.setText(""))
            
        elif (not re.search(regex_pwd, pwd)) or (not re.search(regex_pwd, confPwd)):
                  
            self.Status.setText("") #initial text

            #change text
            QtCore.QTimer.singleShot(100, lambda: self.Status.setText("<strong>* Invalid Password !! *</strong>"))
            QtCore.QTimer.singleShot(3000, lambda: self.Status.setText(""))
            
        elif pwd != confPwd :

            self.Status.setText("") #initial text

            #change text
            QtCore.QTimer.singleShot(100, lambda: self.Status.setText("<strong>* Password do not match *</strong>"))
            QtCore.QTimer.singleShot(3000, lambda: self.Status.setText(""))
        
        else:
            
            con=p.connect(host='localhost',user='root',password=mysql_password,db='analysis',charset='utf8mb4',cursorclass=p.cursors.DictCursor)
            cur=con.cursor()
            q="update sign_up set password='%s' where emailid='%s';"
            cur.execute(q %(pwd,self.email))
            
            q1="update login set password='%s' where emailid='%s';"
            cur.execute(q1 %(pwd ,self.email))
            
            con.commit()
            con.close()
            
            #show signup window
            self.main= LoginWindow()
            self.main.show()

            #close login window
            self.close()
    
    
    def cancel(self):
        
        #show signup window
        self.main= LoginWindow()
        self.main.show()
            
        #close login window
        self.close()
        
        

In [99]:
##Login Window
class LoginWindow(QMainWindow, ui_login.Ui_MainWindow):
    def __init__(self,parent=None):
        QMainWindow.__init__(self)
        super(LoginWindow, self).__init__(parent)
        self.setupUi(self)
        self.setWindowIcon(QtGui.QIcon("robo.gif"))
        self.setFixedSize(917, 585)
        
        #when login or submit button is clicked
        self.Submit_Button.clicked.connect(self.loginFunction)
        self.signup_Button.clicked.connect(self.gotoRegister)
        self.lineEdit_pwd.returnPressed.connect(self.loginFunction) #on pressing Enter key Login
        self.Forgot_pwd.clicked.connect(self.ForgotPwdDialog) #forgot pwd
        
    def loginFunction(self):
        email=self.lineEdit_id.text()
        pwd=self.lineEdit_pwd.text()
        
        if NotValid(email,pwd):
            self.error.setText("") #initial text
            
            #change text
            QtCore.QTimer.singleShot(100, lambda: self.error.setText("<strong> Enter Valid Email-ID or Password !! </strong>"))
            QtCore.QTimer.singleShot(3000, lambda: self.error.setText(""))
        else:
            if not(self.verify(email,pwd)):
                
                self.error.setText("") #initial text

                #change text
                QtCore.QTimer.singleShot(100, lambda: self.error.setText("<strong> Invalid Credentials !! </strong>"))
                QtCore.QTimer.singleShot(3000, lambda: self.error.setText(""))
            else:
                
                self.error.setText("<strong> You are logged in successfully </strong>")
                self.main=UserDashboard()
                self.main.show()

                #close login window
                self.close()
            
            
        
    def gotoRegister(self):
        
        #show signup window
        self.main=RegisterWindow()
        self.main.show()
            
        #close login window
        self.close()
        
    def verify(self,email,pwd):
       
        con=p.connect(host='localhost',user='root',password=mysql_password,db='analysis',charset='utf8mb4',cursorclass=p.cursors.DictCursor)
        cur=con.cursor()
        q="select password from login where emailid='%s'"
        cur.execute(q %(email))
        self.data=cur.fetchone()
       
        con.commit()
        con.close()
        
        if self.data==None:  #email id does not exist
            return False
        else:
            pd=self.data['password']
            if pd!=pwd:
                return False
            else:
                global userID
                userID=email
                history[userID]=''
                return True

    def ForgotPwdDialog(self):
        #show signup window
        self.main=ForgotPassword()
        self.main.show()
        
        self.close()
        
        

In [100]:
##Sign up window
class RegisterWindow(QMainWindow, Ui_SignUpWindow):
    def __init__(self,parent=None):
        QMainWindow.__init__(self)
        super(RegisterWindow, self).__init__(parent)
        self.setupUi(self)
        self.setWindowIcon(QtGui.QIcon("robo.gif"))
        self.setFixedSize(854, 572)
        
        #when signup or submit button is clicked
        self.SignupButton.clicked.connect(self.SignupFunction)
        self.Reset.clicked.connect(self.ResetFunction)
        self.back.clicked.connect(self.Back)
        self.Confirm_pwd.returnPressed.connect(self.SignupFunction)
        
    def SignupFunction(self):
        
        #......................................................
        def insertData(name,email,pwd):
            con=p.connect(host='localhost',user='root',password=mysql_password,db='analysis',charset='utf8mb4',cursorclass=p.cursors.DictCursor)
            cur=con.cursor()
            q="insert into sign_up values('%s','%s','%s')"
            cur.execute(q %(name,email,pwd))
            
            q1="insert into login values('%s','%s')"
            cur.execute(q1 %(email,pwd))
            
            con.commit()
            con.close()
        #.......................................................
        
        email=self.Email_id.text()
        name=self.Name.text()
        pwd=self.Password.text()
        
        regex_name=(r'([a-zA-Z])\D*([a-zA-Z])$')
        
        if self.emailExist(email):
            
            self.label_warning.setText("") #initial text
            
            #change text
            QtCore.QTimer.singleShot(100, lambda: self.label_warning.setText("<strong>* Email-id already exist !! *</strong>"))
            QtCore.QTimer.singleShot(3000, lambda: self.label_warning.setText(""))
        
        elif pwd != self.Confirm_pwd.text() :
            
            self.label_warning.setText("") #initial text
            
            #change text
            QtCore.QTimer.singleShot(100, lambda: self.label_warning.setText("<strong>* Password do not match *</strong>"))
            QtCore.QTimer.singleShot(3000, lambda: self.label_warning.setText(""))
            
        elif not(re.search(regex_name, name)):
            self.label_warning.setText("") #initial text
            
            #change text
            QtCore.QTimer.singleShot(100, lambda: self.label_warning.setText("<strong>*Enter Valid Name *</strong>"))
            QtCore.QTimer.singleShot(3000, lambda: self.label_warning.setText(""))
        
        elif NotValid(email,pwd):
            
            self.label_warning.setText("") #initial text
            
            #change text
            QtCore.QTimer.singleShot(100, lambda: self.label_warning.setText("<strong>* Invalid email-id or password *</strong>"))
            QtCore.QTimer.singleShot(3000, lambda: self.label_warning.setText(""))
            
        else:
            
            #insert data in database
            insertData(name,email,pwd)
            
            #show login window
            self.main=LoginWindow()
            self.main.show()
            
            #close register window
            self.close()
            
    def emailExist(self,email):
        
        con=p.connect(host='localhost',user='root',password=mysql_password,db='analysis',charset='utf8mb4',cursorclass=p.cursors.DictCursor)
        cur=con.cursor()
        q="select 1 from sign_up where exists (select name from sign_up where emailid='%s');"
        cur.execute(q %(email))
        data=cur.fetchone()
        con.commit()
        con.close()
        if data==None:
            return False
        return True
    
    def ResetFunction(self):
        
        self.Name.clear()
        self.Email_id.clear()
        self.Password.clear()
        self.Confirm_pwd.clear()
        
            
    def Back(self):
        
        #show dashboard window
        self.main=LoginWindow()
        self.main.show()
            
        #close video analysis window
        self.close()
        

In [101]:
##Splash Screen
class SplashScreen(QMainWindow, Ui_MainWindow):
    def __init__(self,parent=None):
        QMainWindow.__init__(self)
        super(SplashScreen, self).__init__(parent)
        self.setupUi(self)
        
        #remove title bar
        self.setWindowFlag(QtCore.Qt.FramelessWindowHint)
        self.setAttribute(QtCore.Qt.WA_TranslucentBackground)
        
        #drop shadow effect
        self.shadow = QGraphicsDropShadowEffect(self)
        self.shadow.setBlurRadius(20)
        self.shadow.setXOffset(0)
        self.shadow.setYOffset(0)
        self.shadow.setColor(QColor(0,0,0,60))
        self.frame1.setGraphicsEffect(self.shadow)
        
        # Qtimer
        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(self.progress)
        #timer in millissec
        self.timer.start(60)
        
        #change description
        self.label_loading.setText("<strong>WELCOME</STRONG> User") #initial text
        
        #change text
        QtCore.QTimer.singleShot(1000, lambda: self.label_loading.setText("<strong>LOADING DATABASE</STRONG>"))
        QtCore.QTimer.singleShot(1500, lambda: self.label_loading.setText("<strong>LOADING INTERFACE</STRONG>"))
        
    def progress(self):
        global counter
        
        #set value to progress bar
        self.progressBar.setValue(counter)
        
        #close splash screen and open app
        if counter > 100:
            self.timer.stop()    #stop timer
            
            #show login window
            self.main=LoginWindow()
            self.main.show()
            
            #close splash screen
            self.close()
            
        #increase counter
        counter+=1

In [ ]:
if __name__=="__main__":
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    window=SplashScreen()
    window.show()
    app.exec_()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)

['Theft', 'Theft', 'Theft', 'Theft', 'Theft', 'Theft', 'Theft', 'Theft', 'Theft', 'Theft']
['16/07/2021 18:06:13', '16/07/2021 18:06:15', '16/07/2021 18:06:17', '16/07/2021 18:06:18', '16/07/2021 18:06:20', '16/07/2021 18:06:22', '16/07/2021 18:06:24', '16/07/2021 18:06:26', '16/07/2021 18:06:27', '16/07/2021 18:06:29']
Prakhar
['Theft', 'Theft']
['16/07/2021 18:06:36', '16/07/2021 18:06:38']


NameError: name 'o' is not defined

In [90]:
#pnsvpranjali@gmail.com

In [77]:
#iamafan84@gmail.com